In [1]:
# 크롤링 모듈 import
from wevity_crawler import crawl_wevity
from dacon_crawler import crawl_dacon
from devEvent_crawler import crawl_devEvent

# 각 사이트에서 데이터 크롤링
df_wevity = crawl_wevity()
df_dacon = await crawl_dacon()
df_dev = await crawl_devEvent()

✅ 완료: Jump AI 2025 : 제 3회 AI 신약개발 경진대회
✅ 완료: Boost up AI 2025 : 신약 개발 경진대회
✅ 완료: 2025 Samsung Collegiate Programming Challenge : AI 챌린지
✅ 완료: 제 4회 ETRI 휴먼이해 인공지능 논문경진대회
⛔ 마감 스킵: 2025 KHD (Konyang Health Datathon)
✅ 완료: 데이콘 Basic 스트레스 지수 예측 : 건강 데이터로 마음의 균형을 찾아라!
✅ 완료: 사이버 공격 유형 예측 해커톤: 트래픽 속 위협을 식별하라!
⛔ 마감 스킵: HAI(하이)! - Hecto AI Challenge : 2025 상반기 헥토 채용 AI 경진대회
⛔ 마감 스킵: 갑상선암 진단 분류 해커톤 : 양성과 악성, AI로 정확히 구분하라!
⛔ 마감 스킵: 문맥 기반 문장 순서 예측 AI 경진대회
⛔ 마감 스킵: SW마에스트로 제16기 프롬프트 엔지니어링 대회
⛔ 마감 스킵: 건설용 자갈 암석 종류 분류 AI 경진대회
⛔ 마감 스킵: 2025 Bias-A-Thon : Bias 발견 챌린지 <Track 1>
⛔ 마감 스킵: 2025 Bias-A-Thon : Bias 대응 챌린지 <Track 2>
⛔ 마감 스킵: 신용카드 고객 세그먼트 분류 AI 경진대회


In [2]:
import pandas as pd

# 데이터 통합
df = pd.concat([df_wevity, df_dacon, df_dev], ignore_index=True)
df = df[['title', 'date', 'host', 'url', 'html']]
print(f"총 {len(df)}개 이벤트 통합됨")

총 59개 이벤트 통합됨


In [3]:
# 데이터 전처리를 위한 라이브러리 import
from bs4 import BeautifulSoup
import re
from datetime import datetime

def extract_text_from_html_string(html_string: str) -> str:
    """HTML 문자열에서 텍스트만 추출하는 함수"""
    soup = BeautifulSoup(html_string, 'html.parser')
    # 불필요한 태그들 제거
    for tag in soup(['script', 'style', 'noscript', 'header', 'footer', 'svg', 'img', 'meta', 'link']):
        tag.decompose()
    # 텍스트 추출 및 공백 정리
    text = soup.get_text(separator=' ', strip=True)
    return ' '.join(text.split())

def normalize_date(date_str):
    """날짜를 'YYYY.MM.DD ~ YYYY.MM.DD' 형식으로 통일하는 함수"""
    date_str = str(date_str).strip()
    
    # 패턴 1: 완전한 범위 (2025.06.21 (토) 00:00 ~ 2025.07.06 (일) 23:59)
    pattern1 = r'(\d{4})[.\-\/](\d{1,2})[.\-\/](\d{1,2})\s*(?:\([^)]+\))?\s*(?:\d{2}:\d{2})?\s*~\s*(\d{4})[.\-\/](\d{1,2})[.\-\/](\d{1,2})\s*(?:\([^)]+\))?\s*(?:\d{2}:\d{2})?'
    match1 = re.search(pattern1, date_str)
    if match1:
        y1, m1, d1, y2, m2, d2 = match1.groups()
        try:
            start = datetime(int(y1), int(m1), int(d1))
            end = datetime(int(y2), int(m2), int(d2))
            return f"{start:%Y.%m.%d} ~ {end:%Y.%m.%d}"
        except:
            pass
    
    # 패턴 2: 같은 날 시간 범위 (2025.07.09 (수) 00:00 ~ 23:30)
    pattern2 = r'(\d{4})[.\-\/](\d{1,2})[.\-\/](\d{1,2})\s*(?:\([^)]+\))?\s*\d{2}:\d{2}\s*~\s*\d{2}:\d{2}'
    match2 = re.search(pattern2, date_str)
    if match2:
        y1, m1, d1 = match2.groups()
        try:
            start = datetime(int(y1), int(m1), int(d1))
            return f"{start:%Y.%m.%d} ~ {start:%Y.%m.%d}"
        except:
            pass
    
    # 패턴 3: 기본 날짜 범위 (2025-07-07 ~ 2025-08-25, 2025.07.07 ~ 2025.08.25)
    pattern3 = r'(\d{4})[.\-\/](\d{1,2})[.\-\/](\d{1,2})\s*~\s*(\d{4})[.\-\/](\d{1,2})[.\-\/](\d{1,2})'
    match3 = re.search(pattern3, date_str)
    if match3:
        y1, m1, d1, y2, m2, d2 = match3.groups()
        try:
            start = datetime(int(y1), int(m1), int(d1))
            end = datetime(int(y2), int(m2), int(d2))
            return f"{start:%Y.%m.%d} ~ {end:%Y.%m.%d}"
        except:
            pass
    
    # 매칭되지 않으면 원본 반환
    return date_str

In [4]:
# date 컬럼을 'YYYY.MM.DD ~ YYYY.MM.DD' 형식으로 통일
df['date'] = df['date'].apply(normalize_date)

# html 컬럼에서 텍스트만 추출해 description 컬럼에 저장
df['description'] = df['html'].apply(extract_text_from_html_string)

# html 컬럼 삭제 (용량 절약)
df.drop(columns=['html'], inplace=True)

# 결과를 data.csv 파일로 저장 (UTF-8-sig 인코딩)
df.to_csv("data.csv", index=False, encoding="utf-8-sig")

# 데이터프레임 상위 5개 행 출력하여 결과 확인
df.head()

,title,date,host,url,description
0,제4회 그린 이니셔티브 60초 영화제 공모전,2025.05.13 ~ 2025.08.25,제주MBC,https://www.green60sec.co.kr/,제4회 그린 이니셔티브 60초 영화제 공모전 ■ 공모 부문 가로형 60초 영화(HD...
1,[한국장애인재단] 2025 장애 공감 공모전,2025.06.24 ~ 2025.08.12,한국장애인재단,https://contest.kfpd.org/,2025 장애 공감 공모전 ■ 공모자격 - 장애 공감에 대한 관심 있는 사람 누구나...
2,[오비맥주 크리에이티브 공모전] Creative X Challengers 참가자 모집,2025.06.23 ~ 2025.07.11,오비맥주,https://vo.la/dvqubzU,[오비맥주 크리에이티브 공모전] Creative X Challengers 참가자 모...
3,2025년도 전국 대학생 평화 실천 영상 콘테스트,2025.10.20 ~ 2025.10.24,서울평화상문화재단,http://www.spp.or.kr/kor/apply/contest/,2025년도 전국 대학생 평화 실천 영상 콘테스트 ■ 사업목적 전국 대학생 평화 실...
4,2025 경기 청년기회 여행감독 프로그램 일반과정 참가자 모집,2025.06.23 ~ 2025.07.20,경기관광공사,https://linktr.ee/GTO_travel,"2025 경기 청년기회 여행감독 프로그램 일반과정 참가자 모집 ""최고의 경기를 보여..."
